# Imports

In [ ]:
# !pip install yfinance
# !pip install pmdarima
# !pip install hyperopt
# !pip install xgboost
# !pip install numpy -U

In [2]:
import os
from functools import partial
from datetime import datetime

import typing
from typing import Dict
from typing import Any
from typing import Tuple

import numpy as np
import pandas as pd
import sklearn

import utils
import pipelines
import processing
import evaluate
import predict
import train
import preprocessing

# Download stock daily prices & indexes

In [9]:
### Gets all ticker names (no argument given)
market_cap_min_mm = 5000
market_cap_max_mm = None

ticker_list = utils.get_ticker_names(
    market_cap_min_mm=market_cap_min_mm,
    market_cap_max_mm=market_cap_max_mm,
    remove_industries=[
        "pharmaceuticals_biotechnology_and_life_sciences",
        "health_care_equipment_and_services",
    ],
)

In [10]:
### Specific date - 3rd of March 2022 (Y, M, D)
# date_to = datetime(2021, 1, 18)
### Date of today
date_to = datetime.today()
### How many years' of data to download (going backwards from date_end). Year can be a floating point number
period_years = 8

In [11]:
_, _ = utils.download_stonk_prices(
    ticker_list.index, period_years=period_years, date_to=date_to
)
_, _ = utils.download_stonk_prices(
    ["^VIX"], period_years=period_years, date_to=date_to, fname_prefix="vix"
)
_, _ = utils.download_stonk_prices(
    ["^GSPC"], period_years=period_years, date_to=date_to, fname_prefix="sp500"
)
_, _ = utils.download_stonk_prices(
    ["CL=F"], period_years=period_years, date_to=date_to, fname_prefix="oil"
)
_, _ = utils.download_stonk_prices(
    ["DX=F"], period_years=period_years, date_to=date_to, fname_prefix="usd"
)
_, _ = utils.download_stonk_prices(
    ["^TNX"], period_years=period_years, date_to=date_to, fname_prefix="yield"
)
_, _ = utils.download_stonk_prices(
    ["HG=F"], period_years=period_years, date_to=date_to, fname_prefix="copper"
)

[*********************100%***********************]  1063 of 1063 completed

41 Failed downloads:
- CTXS: No timezone found, symbol may be delisted
- NLSN: No timezone found, symbol may be delisted
- BIP.PRA: No timezone found, symbol may be delisted
- DRE: No timezone found, symbol may be delisted
- LFC: No timezone found, symbol may be delisted
- Y: No timezone found, symbol may be delisted
- HNP: No timezone found, symbol may be delisted
- MIME: No timezone found, symbol may be delisted
- SNP: No timezone found, symbol may be delisted
- FB: No timezone found, symbol may be delisted
- VG: No timezone found, symbol may be delisted
- DIDI: No timezone found, symbol may be delisted
- SAIL: No timezone found, symbol may be delisted
- TMX: No timezone found, symbol may be delisted
- ZEN: No timezone found, symbol may be delisted
- PTR: No timezone found, symbol may be delisted
- CLR: No timezone found, symbol may be delisted
- SNX.WI: No timezone found, symbol may be delisted
- PLAN: No ti

# Run data pipeline

In [12]:
industries = [
    # 'health_care_equipment_and_services',
    "software_and_services",
    "retailing",
    "telecommunication_services",
    "capital_goods",
    "energy",
    # 'pharmaceuticals_biotechnology_and_life_sciences',
    "consumer_staples",
    "banks",
    "diversified_financials",
    "metals_and_mining",
    "technology_hardware_and_equipment",
    "utilities",
    "chemicals",
    "automobiles_and_components",
    "semiconductors_and_semiconductor_equipment",
    "media_and_entertainment",
    "real_estate",
    "consumer_services",
    "consumer_durables_and_apparel",
    "insurance",
    "transportation",
    "commercial_and_professional_services",
    # "paper_and_forest_products",
    "containers_and_packaging",
    # "construction_materials",
]

l_reg = 4
l_roll = 3
dt = 10
last_residual_cutoff = 2.5
adf_pval_cutoff = 0.1
adf_pass_rate_filter = 0.5
mean_max_residual_dt = 21
arima_forecast_months = 3
arima_eval_models = 5

market_cap_max_string = "max" if market_cap_max_mm is None else str(market_cap_max_mm)
pipeline_dir = (
    "pipeline_run_" + str(market_cap_min_mm) + "_to_" + market_cap_max_string + "_cap"
)
output_dir = os.path.join("data", pipeline_dir)

# stonk_model = predict.XGBStonkModel()

market_indexes = utils.get_market_indexes()

In [13]:
datasets = []
i = 0
total_industries = len(industries)
for industry in industries:
    i += 1
    stonks = utils.get_stonk_data(
        market_cap_min_mm,
        market_cap_max_mm,
        remove_industries=[industry],
        filter_industries=True,
    )
    X, Y = processing.combine_stonk_pairs(stonks)

    print("Industry ({0}/{1}): {2}".format(i, total_industries, industry))

    features = pipelines.process_features_from_price_data(
        X=X,
        Y=Y,
        market_indexes=market_indexes,
        l_reg=l_reg,
        l_roll=l_roll,
        dt=dt,
        last_residual_cutoff=last_residual_cutoff,
        adf_pval_cutoff=adf_pval_cutoff,
        adf_pass_rate_filter=adf_pass_rate_filter,
        mean_max_residual_dt=mean_max_residual_dt,
        arima_forecast_months=arima_forecast_months,
        arima_eval_models=arima_eval_models,
    )

    if len(features) == 0:
        print("No trades")
        continue

    print(
        "Mean max value for {0}: {1}".format(industry, features["residuals_max_mean"])
    )
#     dataset = utils.build_dataset_from_live_data_by_industry(
#         std_residuals=features["std_residuals"],
#         adfs=features["adfs"],
#         subindustry=industry,
#         mean_max_residual=features["residuals_max_mean"],
#         residual_quantile=features["residuals_quantile"],
#         vix_index=market_indexes["vix"].loc[stonks.columns[-1]],
#         betas_stability_rsquared=features["beta_stability_rsquared_vals"],
#         arima_forecasts=features["arima_forecasts"],
#     )

#     predictions, df_processed = stonk_model.predict(dataset)
#     datasets.append((dataset, df_processed))
#     predictions = pd.DataFrame(predictions)
#     predictions.index = features["adfs"].index

    features["residuals"].insert(0, "dates", features["dates_index"].values)
    features["betas"].insert(0, "dates", features["dates_index"].values)

    features["residuals"].to_csv(
        os.path.join(output_dir, industry + "_residuals.csv"),
        header=False,
        index=True,
    )
    features["betas"].to_csv(
        os.path.join(output_dir, industry + "_betas.csv"), header=False, index=True
    )
    features["adfs_raw"].to_csv(
        os.path.join(output_dir, industry + "_adfs_raw.csv"), header=False, index=True
    )
    # predictions.to_csv(
    #     os.path.join(output_dir, industry + "_predictions.csv"),
    #     header=False,
    #     index=True,
    # )
    features["arima_forecasts"].to_csv(
        os.path.join(output_dir, industry + "_arima.csv"),
        header=False,
        index=True,
    )
    features["beta_stability_rsquared_vals"].to_csv(
        os.path.join(output_dir, industry + "_rsquared.csv"),
        header=False,
        index=True,
    )
    features["market_correlations"].to_csv(
        os.path.join(output_dir, industry + "_correlations.csv"),
        header=True,
        index=True,
    )

print("*** All done ***")

Industry (1/22): software_and_services
Mean max value for software_and_services: 3.5220000743865967
Industry (2/22): retailing
Mean max value for retailing: 3.053999900817871
Industry (3/22): telecommunication_services
No trades
Industry (4/22): capital_goods
Mean max value for capital_goods: 3.8440001010894775
Industry (5/22): energy
Mean max value for energy: 3.7639999389648438
Industry (6/22): consumer_staples
Mean max value for consumer_staples: 2.630000114440918
Industry (7/22): banks
Mean max value for banks: 3.9010000228881836
Industry (8/22): diversified_financials
Mean max value for diversified_financials: 3.384000062942505
Industry (9/22): metals_and_mining
Mean max value for metals_and_mining: 3.0
Industry (10/22): technology_hardware_and_equipment
Mean max value for technology_hardware_and_equipment: 4.349999904632568
Industry (11/22): utilities
Mean max value for utilities: 3.0810000896453857
Industry (12/22): chemicals
Mean max value for chemicals: 1.7510000467300415
Indu

# Data collection

In [3]:
stonks = utils.get_stonk_data(disable_filter=True)
stonks = stonks.loc[:, :"2022-09-16"]

In [ ]:
pipelines.data_collection_rolling_pipeline(
    stonk_prices=stonks,
    l_reg=3,
    l_roll=2,
    dt=10,
    market_cap_min_mm=1000,
    market_cap_max_mm=None,
    last_residual_cutoff=2.5,
    mean_max_residual_dt=21,
    adf_pval_cutoff=0.1,
    adf_pass_rate_filter=0.5,
    arima_forecast_months=3,
    arima_eval_models=5,
    trade_length_months=3,
    trading_interval_weeks=2,
    remove_industries=[
        "pharmaceuticals_biotechnology_and_life_sciences",
        "health_care_equipment_and_services",
    ],
    first_n_windows=140,
)

Total data windows: 140


In [9]:
dataset = utils.ingest_trade_pipeline_outputs(
    data_dir="data/data_collection_pipeline/1000_to_max"
)

vix = utils.get_stonk_data(fname_prefix="vix", disable_filter=True).iloc[0]
dataset["vix"] = dataset["trade_date"].apply(lambda x: vix.loc[x])

from utils import map_subindustries_to_industries

dataset.loc[:, "industry"] = dataset.apply(map_subindustries_to_industries, axis=1)

dataset.loc[:, "arima_forecast_normalized"] = dataset.apply(
    utils.normalize_arima_forecast, axis=1
)

# new_dataset = []
# for date, date_group in dataset.groupby(by="trade_date"):
#     for industry, industry_group in date_group.groupby(by="subindustry"):
#         residual_quantile = industry_group["last_residual"].abs().quantile(q=0.9)
#         industry_group.loc[:, "residual_quantile"] = np.full(len(industry_group), residual_quantile)
#         new_dataset.append(industry_group)

# new_dataset = pd.concat(new_dataset)

dataset.to_csv("data/dataset_bigcap.csv", header=True, index=False)

# Model development

In [2]:
import xgboost as xgb
from hyperopt import STATUS_OK, STATUS_FAIL, Trials, fmin, hp, tpe, atpe, rand
import pickle

In [13]:
import importlib
importlib.reload(train)

<module 'train' from '/home/jupyter/stonk-rank/train.py'>

In [ ]:
validation_results = pipelines.model_validation_pipeline(
    dataset=df,
    filename_prefix="old_dataset_new_ticker_list_fixed_window",
    fixed_train_window_size=True,
    data_window_max_train_size=60,
    data_window_test_size=2,
    data_window_gap_size=6,
    hp_model_evals=1000,
    top_n_best_trades=5,
    min_industry_confidence=0.4,
    random_noise=0.005,
    hp_nth_best_model=1,
    verbose=False,
)

Total data windows: 33
Period 2019-06-17 to 2022-06-03
  3%|▎         | 32/1000 [00:07<03:55,  4.11trial/s, best loss: -0.3682821436212146]

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]



100%|██████████| 1000/1000 [06:08<00:00,  2.71trial/s, best loss: -0.38095051451406037]
Period 2019-05-17 to 2022-05-06
 34%|███▎      | 336/1000 [01:31<02:31,  4.39trial/s, best loss: -0.6129285963057904]

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]



 39%|███▉      | 393/1000 [01:46<02:17,  4.41trial/s, best loss: -0.6129285963057904]

...

 53%|█████▎    | 527/1000 [02:32<02:22,  3.31trial/s, best loss: -0.30217282165747555]

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]



 54%|█████▎    | 535/1000 [02:35<02:20,  3.32trial/s, best loss: -0.30217282165747555]

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]



 60%|██████    | 600/1000 [02:54<02:06,  3.17trial/s, best loss: -0.30217282165747555]

In [4]:
df = pd.read_csv("data/dataset_bigcap_old.csv")
df = df[df.beta > 0]
df = df[df.last_residual.abs() >= 2.5]
df = preprocessing.assign_labels(df)

In [5]:
updated_ticker_list = utils.get_ticker_names(
    1000,
    None,
    remove_industries=[
        "health_care_equipment_and_services",
        "pharmaceuticals_biotechnology_and_life_sciences",
    ],
)
df = df[df.ticker_x.isin(updated_ticker_list.index)]
df = df[df.ticker_y.isin(updated_ticker_list.index)]

## Production model training

In [55]:
train_size = 60
selected_dates = np.sort(df["trade_date"].unique())[-train_size:]
df_prod = df[df.trade_date.isin(selected_dates)].sample(frac=1)
print(len(df_prod))
print(df_prod["label"].value_counts())

32795
0    26803
1     5992
Name: label, dtype: int64


In [56]:
clf_prod, scalers_prod = train.train_production_xgb(df_prod, params, noise_level=0.005)

[0]	validation_0-logloss:0.68395
[1]	validation_0-logloss:0.67694
[2]	validation_0-logloss:0.67018
[3]	validation_0-logloss:0.66405
[4]	validation_0-logloss:0.65914
[5]	validation_0-logloss:0.65330
[6]	validation_0-logloss:0.64846
[7]	validation_0-logloss:0.64427
[8]	validation_0-logloss:0.64039
[9]	validation_0-logloss:0.63648
[10]	validation_0-logloss:0.63311
[11]	validation_0-logloss:0.63030
[12]	validation_0-logloss:0.62709
[13]	validation_0-logloss:0.62282
[14]	validation_0-logloss:0.61889
[15]	validation_0-logloss:0.61645
[16]	validation_0-logloss:0.61446
[17]	validation_0-logloss:0.61144
[18]	validation_0-logloss:0.60957
[19]	validation_0-logloss:0.60610
[20]	validation_0-logloss:0.60447
[21]	validation_0-logloss:0.60213
[22]	validation_0-logloss:0.60041
[23]	validation_0-logloss:0.59746
[24]	validation_0-logloss:0.59531
[25]	validation_0-logloss:0.59361
[26]	validation_0-logloss:0.59198
[27]	validation_0-logloss:0.58907
[28]	validation_0-logloss:0.58756
[29]	validation_0-loglos

## Model training experiments

In [50]:
import importlib
importlib.reload(preprocessing)

<module 'preprocessing' from '/home/jupyter/stonk-rank/preprocessing.py'>

In [51]:
splits = preprocessing.split_data(
    df, date_count_train=60, date_count_valid=2, date_count_gap=6, random_state=3303544
)
print(len(splits["train"]))
print(len(splits["validation"]))
print(splits["train"]["label"].value_counts())
print(splits["validation"]["label"].value_counts())

30621
964
0    24749
1     5872
Name: label, dtype: int64
0    781
1    183
Name: label, dtype: int64


In [52]:
X_train, scalers = preprocessing.transform_features(splits["train"], noise_level=0.005)

X_valid, _ = preprocessing.transform_features(
    splits["validation"], scalers=scalers, noise_level=0
)

y_train = splits["train"]["label"]
y_valid = splits["validation"]["label"]

In [44]:
df_trial_results = train.model_hp_search(
    df,
    n_evals=1000,
    fixed_train_window_size=False,
    max_train_window_size=68,
    trial_name="new_pipeline#3",
    additive_random_noise=0.005,
    train_window_min_size=60,
    train_window_stride=2,
    write_csv=True,
    random_state=420,
    data_dir="data",
    output_dir="experiments",
)

 72%|███████▎  | 725/1000 [03:32<02:28,  1.85trial/s, best loss: -0.48678320681977716]

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]



100%|██████████| 1000/1000 [05:14<00:00,  3.18trial/s, best loss: -0.48678320681977716]


In [46]:
df_trials = pd.read_csv("data/experiments/new_pipeline#1.csv").sort_values(
    by="ap", ascending=False
)
df_trials.head(10)

,gamma,max_delta_step,max_depth,min_child_weight,n_estimators,scale_pos_weight,f1_score,precision,ap,auc,pos_preds
0,4.098422,1.0,7.0,5.0,48.0,4.578455,0.285714,0.267943,0.536598,0.590195,209
1,2.408549,1.0,7.0,4.0,61.0,4.484318,0.285714,0.281915,0.530729,0.613488,188
2,2.777800,1.0,7.0,4.0,54.0,4.441888,0.276215,0.259615,0.529038,0.611931,208
3,2.124206,1.0,7.0,3.0,50.0,4.366102,0.335106,0.326425,0.527842,0.623108,193
4,3.066145,1.0,7.0,4.0,43.0,4.489866,0.318627,0.288889,0.510161,0.607079,225
5,2.818312,1.0,7.0,4.0,53.0,4.281667,0.293478,0.291892,0.500951,0.620694,185
6,3.905680,1.0,7.0,5.0,48.0,4.451125,0.303665,0.291457,0.498930,0.616279,199
7,2.498228,1.0,7.0,3.0,43.0,4.363691,0.277333,0.270833,0.491842,0.598182,192
8,2.400389,1.0,7.0,3.0,59.0,4.811645,0.304545,0.260700,0.489170,0.609905,257
9,4.990467,1.0,7.0,4.0,59.0,4.448196,0.297650,0.285000,0.486946,0.619634,200


In [30]:
df_trials = pd.read_csv("data/experiments/adf_upgrade_yes-adf-recent#1.csv").sort_values(
    by="ap", ascending=False
)
df_trials.head(10)

,gamma,max_delta_step,max_depth,min_child_weight,n_estimators,scale_pos_weight,train_window_size,f1_score,precision,ap,auc,pos_preds
0,1.802354,3.0,4.0,7.0,64.0,4.104953,64.0,0.276423,0.274194,0.466065,0.615832,186
1,1.708711,3.0,4.0,8.0,63.0,4.365494,64.0,0.298201,0.281553,0.454368,0.628979,206
2,2.033617,3.0,4.0,8.0,62.0,4.208674,64.0,0.293333,0.286458,0.453274,0.628314,192
3,1.160542,3.0,4.0,8.0,69.0,4.151177,56.0,0.275676,0.272727,0.445790,0.615212,187
4,1.291625,3.0,4.0,8.0,65.0,4.340644,64.0,0.305085,0.273913,0.443988,0.637553,230
5,2.506153,3.0,5.0,8.0,70.0,4.382457,64.0,0.323529,0.293333,0.439193,0.639323,225
6,0.846906,3.0,5.0,8.0,30.0,4.048634,64.0,0.295337,0.280788,0.432918,0.624504,203
7,2.397897,3.0,4.0,8.0,41.0,4.015005,64.0,0.317073,0.286344,0.429513,0.620754,227
8,2.164158,3.0,5.0,8.0,46.0,3.958084,64.0,0.293963,0.282828,0.428214,0.637969,198
9,1.959557,3.0,4.0,8.0,74.0,4.835569,64.0,0.298901,0.250000,0.425112,0.618994,272


In [53]:
params = {
    "gamma": 4.098422,
    "scale_pos_weight": 4.578455,
    "max_depth": 7,
    "min_child_weight": 5,
    "max_delta_step": 1,
    "colsample_bylevel": 1,
    "n_estimators": 48,
    "learning_rate": 0.1,
    "subsample": 1,
    "tree_method": "hist",
    "enable_categorical": True,
    "max_cat_to_onehot": 1,
    "eval_metric": ["logloss"],
    "random_state": np.random.randint(999929),
}

clf = xgb.XGBClassifier(**params)
clf = clf.fit(X_train, y_train, eval_set=[(X_valid, y_valid), (X_train, y_train)])

[0]	validation_0-logloss:0.68759	validation_1-logloss:0.68290
[1]	validation_0-logloss:0.68288	validation_1-logloss:0.67338
[2]	validation_0-logloss:0.67600	validation_1-logloss:0.66452
[3]	validation_0-logloss:0.67208	validation_1-logloss:0.65670
[4]	validation_0-logloss:0.66720	validation_1-logloss:0.65007
[5]	validation_0-logloss:0.66452	validation_1-logloss:0.64404
[6]	validation_0-logloss:0.65591	validation_1-logloss:0.63785
[7]	validation_0-logloss:0.65363	validation_1-logloss:0.63217
[8]	validation_0-logloss:0.65215	validation_1-logloss:0.62744
[9]	validation_0-logloss:0.64749	validation_1-logloss:0.62293
[10]	validation_0-logloss:0.64431	validation_1-logloss:0.61923
[11]	validation_0-logloss:0.64278	validation_1-logloss:0.61585
[12]	validation_0-logloss:0.64060	validation_1-logloss:0.61290
[13]	validation_0-logloss:0.63685	validation_1-logloss:0.60929
[14]	validation_0-logloss:0.63028	validation_1-logloss:0.60535
[15]	validation_0-logloss:0.62329	validation_1-logloss:0.60211
[1

In [54]:
print("**Validation**")
y_score = clf.predict_proba(X_valid)[:, 1]
thres = 0.5
y_preds = y_score > thres

df_results_valid = splits["validation"].copy()
df_results_valid["score"] = y_score
df_results_valid["prediction"] = y_preds

_ = evaluate.performance_summary(
    y_score=y_score, y_preds=y_preds, y_true=y_valid, auc_cutoff=0.5, verbose=True
)

_ = evaluate.returns_on_predictions(df_results_valid, verbose=True)

# _ = evaluate.performance_on_slice(df_results_valid, "subindustry")

**Validation**
Precision: 0.245136186770428
PR-AUC/AP score: 0.38282887033811264
ROC-AUC score: 0.5980283089495743
Total positive predictions: 257
Total positive labels: 183

Totals:
        prediction
result            
FN             120
FP             194
TN             587
TP              63

Means:
        return_one_month  return_two_month  return_three_month
result                                                        
FN              0.075925          0.093075            0.080333
FP              0.006701         -0.009912           -0.009582
TN              0.004467         -0.019247           -0.029538
TP              0.066444          0.117079            0.109556

Std:
        return_one_month  return_two_month  return_three_month
result                                                        
FN              0.070681          0.062062            0.076462
FP              0.039699          0.060385            0.071674
TN              0.048008          0.065300            0.069

In [46]:
# pd.set_option("display.max_rows", 200)
# evaluate.performance_on_trading_use_case(
#         df_results_valid, top_n_trades=5, min_industry_score=0.4
#     )

In [31]:
for name, importance in zip(clf.feature_names_in_, clf.feature_importances_):
    print(name, importance)

adf_pass_rate 0.029732222
last_residual 0.08531821
residual_mean_max 0.0812026
industry 0.118948996
vix 0.26275504
betas_rsquared 0.14753708
arima_forecast_normalized 0.21244633
residual_inter 0.062059514


In [44]:
pd.set_option("display.max_rows", 200)

In [ ]:
df_results_valid[df_results_valid.result == "FP"].iloc[:].drop(
    columns=["beta", "intercept", "data_window_start", "label", "prediction"]
).iloc[:100]

In [ ]:
# df_results_valid[df_results_valid.return_three_month < -0.2]

In [ ]:
# df_results_valid[df_results_valid.subindustry == 'consumer_services'].iloc[0:100]

In [3]:
vld = pd.read_csv(
    "data/experiments/validation/final_one_validation_pipeline_60_True_2_6_1000_5_0.4_0.005_1.csv"
)
vld[
    [
        "_pos_pred_ret_3mo",
        "_fp_ret_3mo",
        "_ap",
        "banksbanks_top5_ret_3mo",
        "capitgoods_top5_ret_3mo",
        "chemiicals_top5_ret_3mo",
        "divercials_top5_ret_3mo",
        "energnergy_top5_ret_3mo",
        "semicpment_top5_ret_3mo",
        "softwvices_top5_ret_3mo",
        "technpment_top5_ret_3mo",
        "transation_top5_ret_3mo",
        "utiliities_top5_ret_3mo",
    ]
].describe().drop(index=["std", "count"])

,_pos_pred_ret_3mo,_fp_ret_3mo,_ap,banksbanks_top5_ret_3mo,capitgoods_top5_ret_3mo,chemiicals_top5_ret_3mo,divercials_top5_ret_3mo,energnergy_top5_ret_3mo,semicpment_top5_ret_3mo,softwvices_top5_ret_3mo,technpment_top5_ret_3mo,transation_top5_ret_3mo,utiliities_top5_ret_3mo
mean,-0.000545,-0.041131,0.324907,0.010408,-0.012968,0.021529,-0.026084,0.014053,0.000707,-0.006556,0.021094,-0.004161,0.009587
min,-0.112000,-0.134000,0.044000,-0.132000,-0.228000,-0.262000,-0.308000,-0.255000,-0.293000,-0.136000,-0.181000,-0.201000,-0.089000
25%,-0.024500,-0.056000,0.205000,-0.012750,-0.056000,-0.040000,-0.063500,-0.027000,-0.049500,-0.034000,-0.023500,-0.042500,-0.019250
50%,0.002000,-0.032000,0.274000,0.006000,-0.004000,0.020000,-0.017000,0.012000,0.015000,-0.009000,0.010000,0.002000,0.005000
75%,0.016000,-0.021500,0.397000,0.029500,0.024750,0.054000,0.017000,0.042000,0.045500,0.019500,0.036250,0.041500,0.042000
max,0.164000,0.038000,1.000000,0.176000,0.203000,0.376000,0.146000,0.229000,0.180000,0.138000,0.359000,0.126000,0.104000


In [37]:
vld = pd.read_csv(
    "data/experiments/validation/new_adf_recent_adf_included_validation_pipeline_64_False_2_6_1000_5_0.4_0.005_1.csv"
)
vld[
    [
        "_pos_pred_ret_3mo",
        "_fp_ret_3mo",
        "_ap",
        "banksbanks_top5_ret_3mo",
        "capitgoods_top5_ret_3mo",
        "chemiicals_top5_ret_3mo",
        "divercials_top5_ret_3mo",
        "energnergy_top5_ret_3mo",
        "semicpment_top5_ret_3mo",
        "softwvices_top5_ret_3mo",
        "technpment_top5_ret_3mo",
        "transation_top5_ret_3mo",
        "utiliities_top5_ret_3mo",
    ]
].describe().drop(index=["std", "count"])

,_pos_pred_ret_3mo,_fp_ret_3mo,_ap,banksbanks_top5_ret_3mo,capitgoods_top5_ret_3mo,chemiicals_top5_ret_3mo,divercials_top5_ret_3mo,energnergy_top5_ret_3mo,semicpment_top5_ret_3mo,softwvices_top5_ret_3mo,technpment_top5_ret_3mo,transation_top5_ret_3mo,utiliities_top5_ret_3mo
mean,-0.009548,-0.046613,0.319315,-0.002054,-0.01575,0.017282,-0.030595,0.031155,0.004512,-0.010848,0.031333,-0.012696,0.010837
min,-0.485000,-0.597000,0.062000,-0.095000,-0.21300,-0.166000,-0.273000,-0.217000,-0.305000,-0.112000,-0.181000,-0.188000,-0.093000
25%,-0.030000,-0.052000,0.209000,-0.024000,-0.04975,-0.030000,-0.057000,-0.024750,-0.024250,-0.044000,-0.024500,-0.048500,-0.016750
50%,0.000000,-0.036000,0.292000,-0.004000,-0.00200,0.023000,-0.023000,0.026500,0.010000,-0.011000,0.010000,-0.014000,-0.000500
75%,0.016000,-0.020000,0.391000,0.021000,0.03000,0.055000,0.010250,0.071250,0.054500,0.016500,0.048500,0.047000,0.038250
max,0.112000,0.020000,0.773000,0.102000,0.17600,0.194000,0.065000,0.330000,0.139000,0.177000,1.178000,0.114000,0.151000


In [36]:
vld = pd.read_csv("data/experiments/validation/dynamic-train-window-sparse_validation_pipeline_62_False_2_6_1000_5_0.4_0.005_1.csv")
vld[["_pos_pred_ret_3mo", "_fp_ret_3mo", "_ap", "banksbanks_top5_ret_3mo", "capitgoods_top5_ret_3mo", "chemiicals_top5_ret_3mo", "divercials_top5_ret_3mo", "energnergy_top5_ret_3mo", "semicpment_top5_ret_3mo", "softwvices_top5_ret_3mo", "technpment_top5_ret_3mo", "transation_top5_ret_3mo", "utiliities_top5_ret_3mo"]].describe().drop(index=["std", "count"])

,_pos_pred_ret_3mo,_fp_ret_3mo,_ap,banksbanks_top5_ret_3mo,capitgoods_top5_ret_3mo,chemiicals_top5_ret_3mo,divercials_top5_ret_3mo,energnergy_top5_ret_3mo,semicpment_top5_ret_3mo,softwvices_top5_ret_3mo,technpment_top5_ret_3mo,transation_top5_ret_3mo,utiliities_top5_ret_3mo
mean,0.047659,-0.027708,0.35029,0.004711,-0.002697,0.010904,-0.004976,0.02812,-0.002802,0.000348,0.019851,0.006367,0.006594
min,-0.094000,-0.101000,0.00000,-0.104000,-0.190000,-0.106000,-0.146000,-0.19000,-0.226000,-0.271000,-0.675000,-0.218000,-0.064000
25%,-0.020250,-0.046250,0.16700,-0.026750,-0.048000,-0.028000,-0.043250,-0.04200,-0.030000,-0.032000,-0.027500,-0.034000,-0.020000
50%,0.015500,-0.021000,0.32100,-0.002500,-0.002000,0.001000,-0.002500,0.00300,0.000000,0.003000,0.032000,0.011500,0.005000
75%,0.044250,-0.003000,0.47300,0.030500,0.035000,0.047000,0.033000,0.06750,0.048000,0.053000,0.077000,0.040500,0.020000
max,1.309000,0.036000,1.00000,0.153000,0.261000,0.162000,0.136000,0.55800,0.138000,0.164000,0.824000,0.228000,0.088000
